In [89]:
from pickle import FALSE
from networkx import Graph
import networkx as nx
import pandas as pd
from pathlib import Path

import opendssdirect as dss


class DistNetwork(Graph):

    def Add_Nodes(self, node_csv):
        nodes_df = pd.read_csv(node_csv)
        for bus in nodes_df.index:
            bus_name = nodes_df['Bus'][bus]
            Loc_x = nodes_df['Graph_Loc_x'][bus]
            Loc_y = nodes_df['Graph_Loc_y'][bus]
            graph_coords = (Loc_x, Loc_y)
            self.add_node(bus_name, coords = graph_coords)
    
    def Add_LineGeoms(self, lineGeoms_csv):
        line_geoms_df = pd.read_csv(lineGeoms_csv, 
                                    index_col='LineGeo', 
                                    keep_default_na=FALSE
                                    )
        self.line_geoms = line_geoms_df

    def get_network_wires(self) -> set:
        OH_line_geos = self.line_geoms['Type']=='OH'
        n_cond_select = self.line_geoms['Phases'] != self.line_geoms['Conductors']
        ph_conductors = set(self.line_geoms['phase_conductor'][OH_line_geos])
        n_conductors = set(self.line_geoms['neutral_conductor'][n_cond_select])
        return n_conductors.union(ph_conductors)

    def get_network_cnwires(self) -> set:
        CN_line_geos = self.line_geoms['Type']=='CN'
        cn_conductors = set(self.line_geoms['neutral_conductor'][CN_line_geos])
        return cn_conductors
    
    def Add_WireData(self, wireData_csv):
        columns = ['Type',
                   'GMRac',
                   'GMRunits',
                   'rac',
                   'runits',
                   'normamps'
                   ]
        wire_data_df = pd.read_csv(wireData_csv, index_col='Type', usecols=columns)
        wires = wire_data_df.index
        wset = wires.isin(self.get_network_wires())
        self.wire_data = wire_data_df.loc[wset]

    def Add_CNData(self, CNData_csv):
        columns = ['Type',
                   'GMRac',
                   'GmrStrand',
                   'GMRunits',
                   'Rac',
                   'Rstrand',
                   'Runits',
                   'DiaCable',
                   'DiaIns',
                   'DiaStrand',
                   'diam',
                   'Radunits',
                   'normamps',
                   'k'
                   ]
        CN_data_df = pd.read_csv(CNData_csv, index_col='Type', usecols=columns)
        cn_wires = CN_data_df.index
        cnset = cn_wires.isin(self.get_network_cnwires())
        self.CN_data = CN_data_df.loc[cnset]

    def Add_Lines(self, lines_csv):
        lines_df = pd.read_csv(lines_csv)
        for line in lines_df.index:
            Bus1 = lines_df['bus1'][line]
            Bus2 = lines_df['bus2'][line]
            length = lines_df['Length'][line]
            units = lines_df['Units'][line]
            line_geo = lines_df['LineGeometry']
            self.add_edge(Bus1, 
                          Bus2, 
                          Length = length, 
                          Units = units, 
                          LineGeo = line_geo
                          )
    
    def Add_Loads(self, loads_csv):
        loads_df = pd.read_csv(loads_csv)
        for load in loads_df.index:
            bus = loads_df['Node'][load]
            load_kW = loads_df['kW'][load]
            load_kVAr = loads_df['kVAr'][load]
            self.nodes[bus]["Load_kW"] = load_kW
            self.nodes[bus]["Load_kVAr"] = load_kVAr
    
    def add_network_components(self, component_csvs: dict):
        bus_csv = Path('..', 'network-data', component_csvs['nodes'])
        self.Add_Nodes(bus_csv)
        
        lineGeo_csv = Path('..', 'network-data', component_csvs['line_geos'])
        self.Add_LineGeoms(lineGeo_csv)
        
        wireData_csv = Path('..', 'network-data', component_csvs['wire_data'])
        self.Add_WireData(wireData_csv)
        CNData_csv = Path('..', 'network-data', component_csvs['CN_data'])
        self.Add_CNData(CNData_csv)
        
        lines_csv = Path('..', 'network-data', component_csvs['lines'])
        self.Add_Lines(lines_csv)
        
        loads_csv = Path('..', 'network-data', component_csvs['loads'])
        self.Add_Loads(loads_csv)


    def clear_DSS(self):
        reply = dss.utils.run_command("clear")
        print(reply)

    def solve_DSS(self):
        reply = dss.utils.run_command("solve")
        print(reply)

    def compile_DSS(self):
        self.clear_DSS()
        
        cmd = dss.utils.run_command

        #compile wriedata
        for wire in self.wire_data.index:
            dss_str = 'new wiredata.' + wire + ' '
            for attr in self.wire_data.columns:
                dss_str += ' ' + attr + '=' + str(self.wire_data[attr][wire])
            reply = cmd(dss_str)
            print(reply)
        
        #compile CNdata
        for cnwire in self.CN_data.index:
            dss_str = 'new cndata.' + cnwire + ' '
            for attr in self.CN_data.columns:
                dss_str += ' ' + attr + '=' + str(self.CN_data[attr][wire])
            reply = cmd(dss_str)
            print(reply)


    def record_DSS_output(self, class_name):
        class_DSS_output = dss.utils.class_to_dataframe(class_name=class_name)
        self.class_name = class_DSS_output
        return class_DSS_output

    





In [92]:
def main():
    Dnetwork = DistNetwork()

    components = {'nodes': 'nodes.csv',
              'line_geos': 'LineGeometry.csv',
              'wire_data': 'WireData.csv',
              'CN_data': 'CNData.csv',
              'lines': 'lines.csv',
              'loads': 'loads.csv'}
    
    Dnetwork.add_network_components(components)

main()
